# Ikeda $B_e$ assumtion.
Using analytical linear decay solution to calculate the $B_e$

In [ ]:
from rolldecayestimators import equations

# Purpose
The quadratic or cubic model can be expressed using the linearized equivalent damping ($B_e$) according to <cite data-cite="7505983/EYEMHSYH">. But I have some doubt about the validity of this, which will be investigated in this notebook.

In [ ]:
equations.B_e_equation

# Methodology
The linear equivalent damping is basically about replacing a higher order model with a linear model. In this notebook a quadratic model is representing the higher order model.
The quadratic model has two damping coefficients $B_1$ and $B_2$ these to coefficients should be replaced by only one linear damping term $B_e$. <cite data-cite="7505983/FB64RGPF"></cite> suggests that the equation above can be used to combine $B_1$ and $B_2$ into one coefficient $B_e$.

I will use the least square fitting to find the linear equivalent damping that minimized the error when representing a quadratic model as a linear model. The analytical solution to the linear roll decay model will be used to get a fast and accurate prediction of the least square fit.

# WIP - improvements
(WORK IN PROGRESS)
Use this section only if the notebook is not final.

Notable TODOs:
* todo 1
* todo 2
* todo 3

## Results
Describe and comment the most important results.

# Suggested next steps
State suggested next steps, based on results obtained in this notebook.

# Setup

In [ ]:
# %load imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import OrderedDict
#plt.style.use('paper')

#import data
import copy
from mdldb.run import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

from sklearn.metrics import r2_score
from src.data import database
from mdldb import tables


In [ ]:
from rolldecayestimators.simplified_ikeda_class import SimplifiedIkeda
import rolldecayestimators
from scipy.integrate import solve_ivp
from scipy.optimize import least_squares

## Quadratic

In [ ]:
Math(vlatex(equations.roll_decay_equation_himeno_quadratic_b))

In [ ]:
eq_acceleration_quadratic = sp.Eq(symbols.phi_dot_dot,
                        sp.solve(equations.roll_decay_equation_himeno_quadratic_b,symbols.phi_dot_dot)[0])

accelaration_quadratic_lambda = lambdify(sp.solve(equations.roll_decay_equation_himeno_quadratic_b,symbols.phi_dot_dot)[0])

Math(vlatex(eq_acceleration_quadratic))

In [ ]:
class RollDecayQuadratic():
    
    def __init__(self,A_44, B_1, B_2, C_1):
        self.parameters = {
            'A_44':A_44,
            'B_1':B_1,
            'B_2':B_2,
            'C_1':C_1,
        }
            
    def time_step(self,t,states):
        
        phi = states[0]
        phi1d = states[1]
        phi2d = accelaration_quadratic_lambda(**self.parameters, phi=phi, phi1d=phi1d)
        
        d_states_dt = np.array([phi1d, phi2d])
        return d_states_dt
    
    def simulate(self,t,phi0=np.deg2rad(10),phi1d0=0):
        
        initial_state = [phi0,phi1d0]
        
        t_span = [t[0], t[-1]]
        
        result = solve_ivp(fun=simulation.time_step, t_span=t_span,  y0=initial_state, t_eval=t)
        assert result.success
        df_result = pd.DataFrame(index=result.t, data=result.y.T, columns = ['phi','phi1d'])
        return df_result


## Select suitable paramter range

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.95, 
                             exclude_table_name='rolldecay_exclude')

In [ ]:
df_rolldecay.head()

In [ ]:
interesting = ['B_1A','B_2A','C_1A']
df_rolldecay[interesting].describe()

In [ ]:
B_1A = df_rolldecay['B_1A'].median()
B_1A

In [ ]:
index = (df_rolldecay['B_1A']-B_1A).abs().argmin()
row = df_rolldecay.iloc[index]

In [ ]:
row['B_1A']

In [ ]:
B_max_ratio = (df_rolldecay['B_2A']/df_rolldecay['B_1A']).abs().quantile(0.95)

In [ ]:
N=100000
A_44 = 2.2
B_1 = row['B_1A']*A_44
C_1 = row['C_1A']*A_44

simulations = {
}

N= 15
B_2_max = B_max_ratio*B_1
B_2s = np.linspace(0,B_2_max,N)
for B_2 in B_2s:
    simulations[B_2]=RollDecayQuadratic(A_44=A_44, B_1=B_1, B_2=B_2, C_1=C_1)

In [ ]:
B_1

In [ ]:
C_1

In [ ]:
equations.C_equation_linear

In [ ]:
A_44_eq = sp.Eq(symbols.A_44, equations.A44)
A_44_eq

In [ ]:
eqs = [
    A_44_eq,
    equations.C_equation_linear,

]
omega0_eq = sp.Eq(symbols.omega0,sp.solve(eqs, symbols.omega0, symbols.GM)[1][0])
omega0_eq

In [ ]:
omega0 = np.sqrt(C_1/A_44)
N_oscillations=10
t = np.arange(0,2*np.pi/omega0*N_oscillations,0.01)
phi0=np.deg2rad(10)
phi1d0 = 0
initial_state = [phi0,phi1d0]


results = {}
X_amplitudes = {}
for name,simulation in simulations.items():
    
    df_result = simulation.simulate(t=t, phi0=phi0, phi1d0=phi1d0)
    
    results[name]=df_result
    X_amplitudes[name]=rolldecayestimators.measure.calculate_amplitudes_and_damping(X=df_result)

In [ ]:
#for name in results.keys():
#    fig,ax=plt.subplots()
#    df_result = results[name]
#    amplitudes = X_amplitudes[name]
#    df_result.plot(y='phi',ax=ax)
#    amplitudes.plot(y='phi_a', ax=ax)
#    ax.grid(True)
#    ax.set_title(name)

## $B_e$ from anayltica solution

In [ ]:
Math(vlatex(equations.diff_eq))

In [ ]:
Math(vlatex(equations.analytical_solution))

In [ ]:
no_initial_speed = sp.simplify(equations.analytical_solution.subs(symbols.phi_0_dot,0))
Math(vlatex(no_initial_speed))

In [ ]:
no_initial_speed_zeta_small = no_initial_speed.subs(
    [(sp.sqrt(1-symbols.zeta**2),1),
     (symbols.zeta*sp.sin(symbols.omega0*symbols.t),0),
    ])
Math(vlatex(no_initial_speed_zeta_small))

In [ ]:
B_1_zeta_eq = sp.Eq(symbols.B_1, 2*symbols.zeta*symbols.omega0*symbols.A_44)
B_1_zeta_eq

In [ ]:
eqs = [
    B_1_zeta_eq,    
    equations.analytical_solution,
      ]

analytical_solution_B_1 = sp.Eq(symbols.phi,
                                sp.simplify(sp.solve(eqs,symbols.zeta,symbols.phi)[0][1]))
analytical_solution_B_1

In [ ]:
analytical_lambda = lambdify(sp.solve(analytical_solution_B_1,symbols.phi)[0])

In [ ]:
equations.extinction_equation

In [ ]:
sp.Eq(symbols.zeta,sp.solve(equations.extinction_equation,symbols.zeta)[0])

In [ ]:
def simulate(B_e,X,omega0,A_44):
    t = X.index
    initial_states = X.iloc[0]
    phi_0 = initial_states['phi']
    phi_01d= initial_states['phi1d']
    phi_pred = analytical_lambda(omega0=omega0, A_44=A_44, phi_0=phi_0, phi_01d=phi1d0, t=t, B_1=B_e)
    return phi_pred

def residuals(B_e,X,omega0,A_44):
    
    phi_pred = simulate(B_e=B_e, X=X, omega0=omega0, A_44=A_44)
    residual = phi_pred - X['phi']
    return residual

In [ ]:
def linear_equivalent(X,omega0,A_44):
    kwargs={
        'X':X,
        'omega0':omega0,
        'A_44':A_44,
    }
    initial_guess = [B_1]
    result = least_squares(fun=residuals, x0=initial_guess, kwargs=kwargs, method='lm')
    assert result.success is True
    B_e = result.x[0]
    return B_e

In [ ]:
X = results[0]
omega0 = np.sqrt(C_1/A_44)

B_e = linear_equivalent(X=X, omega0=omega0, A_44=A_44)
phi_pred = simulate(B_e=B_e, X=X, omega0=omega0, A_44=A_44)
phi_error = X['phi'] - phi_pred

fig,ax=plt.subplots()
X.plot(y='phi',ax=ax)
ax.plot(t,phi_pred,'--')

r2_score(y_true=X['phi'], y_pred=phi_pred)

In [ ]:
df_B_e = pd.DataFrame()
df_B_e.index.name='B_2'

for B_2, X in results.items():
    B_e = linear_equivalent(X=X, omega0=omega0, A_44=A_44)
    df_B_e.loc[B_2,'B_e'] = B_e
    phi_pred = simulate(B_e=B_e, X=X, omega0=omega0, A_44=A_44)
    df_B_e.loc[B_2,'R2'] = r2_score(y_true=X['phi'], y_pred=phi_pred)
    
    B_e_himeno = lambdas.B_e_lambda(B_1=B_1, B_2=B_2, omega0=omega0, phi_a=phi0)
    df_B_e.loc[B_2,'B_e_himeno'] = B_e_himeno
    phi_pred_himeno = simulate(B_e=B_e_himeno, X=X, omega0=omega0, A_44=A_44)
    df_B_e.loc[B_2,'R2_himeno'] = r2_score(y_true=X['phi'], y_pred=phi_pred_himeno)

df_B_e['B_1'] = B_1

In [ ]:
fig,ax=plt.subplots()
df_B_e.plot(y=['B_1','B_e','B_e_himeno'], ax=ax)
ax.set_ylabel('$B_e$ [Nm/s] linear equivalent damping')

fig,ax=plt.subplots()
df_B_e.plot(y=['R2','R2_himeno'],ax=ax)
ax.set_ylabel('$R^2$ [-] coefficient of determination')

## $B_2/2$

In [ ]:
df_B_e = pd.DataFrame()
df_B_e.index.name='B_2'

for B_2, X in results.items():
    B_e = linear_equivalent(X=X, omega0=omega0, A_44=A_44)
    df_B_e.loc[B_2,'B_e'] = B_e
    phi_pred = simulate(B_e=B_e, X=X, omega0=omega0, A_44=A_44)
    df_B_e.loc[B_2,'R2'] = r2_score(y_true=X['phi'], y_pred=phi_pred)
    
    B_e_himeno = lambdas.B_e_lambda(B_1=B_1, B_2=B_2/2, omega0=omega0, phi_a=phi0)
    df_B_e.loc[B_2,'B_e_himeno'] = B_e_himeno
    phi_pred_himeno = simulate(B_e=B_e_himeno, X=X, omega0=omega0, A_44=A_44)
    df_B_e.loc[B_2,'R2_himeno'] = r2_score(y_true=X['phi'], y_pred=phi_pred_himeno)

df_B_e['B_1'] = B_1

In [ ]:
fig,ax=plt.subplots()
df_B_e.plot(y=['B_1','B_e','B_e_himeno'], ax=ax)
ax.set_ylabel('$B_e$ [Nm/s] linear equivalent damping')

fig,ax=plt.subplots()
df_B_e.plot(y=['R2','R2_himeno'],ax=ax)
ax.set_ylabel('$R^2$ [-] coefficient of determination')